In [1]:
import numpy as np
qubit = np.asarray([[1,0],[0,1]])

## Qubit generator

In [2]:
class Qubit():
    name = 'Qubit Object'
    def __init__(self,num_qubit=1,*state):         
        if state == ():
            print("Please specifies qubit state! Example for two-qubits state 00: [0,0]")
        else:
            self.num_qubit = num_qubit
            self.state = state
            self.matrix = Qubit.generate(self)
        
    def generate(self):
        global qubit
        obj = qubit[[self.state[0][0]]]
        if(len(self.state[0])>1):
            for i in range(1,len(self.state[0])):
                obj = np.tensordot(obj,qubit[self.state[0][i]],0)
        return obj.reshape(2**self.num_qubit,1)    

## Quantum Gate

In [58]:
class Gate:
    def __init__(self,obj):
        try:
            test = obj.name
            self.obj = obj
            self.obj_m = obj.matrix
        except:
            print("Argument must be quantum object")

### Single-qubit gate

In [59]:
class Pauli(Gate):
    sigmaX_ = np.array([[0,1],[1,0]])
    sigmaY_ = np.array([[0,-1j],[1j,0]])
    sigmaZ_ = np.array([[1,0],[0,-1]])
    
    def sigmaX(self):
        result = np.dot(np.array([[0,1],[1,0]]),self.matrix)
        self.matrix = result
        for i in range(len(qubit)):
            if np.all(self.matrix== qubit[i].reshape(2,1)):
                self.state[0][0] = i
        return self.matrix
    
    def sigmaY(self):
        result = np.dot(np.array([[0,-1j],[1j,0]]),self.matrix)
        self.matrix = result
        for i in range(len(qubit)):
            if np.all(self.matrix== qubit[i].reshape(2,1)):
                self.state[0][0] = i
        return self.matrix
    
    def sigmaZ(self):
        result = np.dot(np.array([[1,0],[0,-1]]),self.matrix)
        self.matrix = result
        for i in range(len(qubit)):
            if np.all(self.matrix== qubit[i].reshape(2,1)):
                self.state[0][0] = i
        return self.matrix

In [125]:
class I(Gate):
    M = np.identity(2).astype(int)
    def act(self):
        result = np.dot(np.identity(2),self.matrix).astype(int)
        self.matrix = result
        for i in range(len(qubit)):
            if np.all(self.matrix== qubit[i].reshape(2,1)):
                self.state[0][0] = i
        return self.matrix

In [100]:
class U_Matrix(Gate):
    def X(theta):
        X = np.identity(2).astype(int)*np.cos(theta/2) - 1j*np.sin(theta/2)*Pauli.sigmaX_
        return X
    def Y(theta):
        Y = np.identity(2).astype(int)*np.cos(theta/2) - 1j*np.sin(theta/2)*Pauli.sigmaY_
        return Y
    def Z(theta):
        Z = np.identity(2).astype(int)*np.cos(theta/2) - 1j*np.sin(theta/2)*Pauli.sigmaZ_
        return Z
    
    def act(self,var,theta):    
        if var=='X':
            opr = Pauli.sigmaX_
        elif var=='Y':
            opr = Pauli.sigmaY_
        else:
            opr = Pauli.sigmaZ_
        matrix = np.identity(2)*np.cos(theta/2) - 1j*(np.sin(theta/2))*opr
        result = np.dot(matrix,self.matrix)
        self.matrix = result.astype(int)
        return self.matrix

In [101]:
class Phase(Gate):
    def M(theta):
        matrix = np.array([[1,0],[0,np.cos(theta)+1j*np.sin(theta)]])
        return matrix
    
    def act(self,theta):
        matrix = Phase.Phase_M(theta)
        result = np.dot(matrix,self.matrix)
        self.matrix = result
        return self.matrix

In [102]:
class H(Gate):
    M = np.dot(Pauli.sigmaX_,U_Matrix.Y(np.pi/2))
    
    def act(self):
        result = np.dot(H.M,self.matrix)
        self.matrix = result
        return self.matrix

### Two-qubit gate

In [78]:
class CP(Gate):
    def M(theta=0):
        phi = np.cos(theta) + 1j*np.sin(theta)
        matrix = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,phi]]) 
        return matrix.astype(int)
    
    def act(self,theta=0):
        result = np.dot(CP.M(theta),self.matrix)
        self.matrix = result
        return self.matrix

In [86]:
class CNOT(Gate):
    M = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
    def act(self):
        result = np.dot(CNOT.M,self.matrix)
        self.matrix = result
        return self.matrix

### Three-qubit gate

In [90]:
class Toffoli(Gate):
    matrix = np.identity(8)
    matrix[6,7] = matrix[7,6] = 1
    matrix[6,6] = matrix[7,7] = 0
    M = matrix
    
    def act(self):
        result = np.dot(Toffoli.M,self.matrix)
        self.matrix = result
        return self.matrix

In [94]:
class Fredkin(Gate):
    matrix = np.identity(8)
    matrix[5,6]=matrix[6,5]=1
    matrix[5,5]=matrix[6,6]=0
    M = matrix
    
    def act(self):
        result = np.dot(Fredkin.M,self.matrix)
        self.matrix = result
        return self.matrix

## Bell States Construction

In [99]:
q = Qubit(2,[0,0])

In [157]:
q1 = qubit[q.state[0][0]].reshape(2,1)
H1 = np.dot(H.M,q1)
qs = np.tensordot(H1,qubit[q.state[0][1]].reshape(2,1),0).reshape(4,-1)
bell_state = np.real(np.dot(CNOT.M,qs))

In [158]:
bell_state 

array([[ 0.70710678],
       [ 0.        ],
       [ 0.        ],
       [ 0.70710678]])